In [1]:
import bluprint
from bluprint import cli

bp = cli.Bluprint()
bp.create(
	project_name='project',
	python_version='>=3.11.2',
		parent_dir='/home/igor/tmp',
)

Bluprint: creating Python project... 

>> pyenv local



The specified Python version (3.11.6) is not supported by the project (3.11.2).
Please choose a compatible version or loosen the python constraint specified in the pyproject.toml file.
pyenv: shell integration not enabled. Run `pyenv init' for instructions.
The currently activated Python version 3.8.10 is not supported by the project (3.11.2).
Trying to find and use a compatible version. 

Poetry was unable to find a compatible version. If you have one, you can explicitly use it via the "env use" command.
The currently activated Python version 3.8.10 is not supported by the project (3.11.2).
Trying to find and use a compatible version. 

Poetry was unable to find a compatible version. If you have one, you can explicitly use it via the "env use" command.
ok


In [2]:
import subprocess

subprocess.run('pyenv versions', shell=True, cwd='/home/igor/tmp')

  system
  3.9.8
  3.10.10
* 3.11.2 (set by /home/igor/.pyenv/version)
  3.11.6
  3.12.0
  3.12.0a5
  3.13.0a2


CompletedProcess(args='pyenv versions', returncode=0)

In [4]:
subprocess.run(['pyenv', 'global', '3.11.6'])

CompletedProcess(args=['pyenv', 'global', '3.11.6'], returncode=0)

In [5]:
from importlib import resources

In [9]:
resources.files('tests.yaml')._paths[0]

PosixPath('/home/igor/repos/bluprint/tests/yaml')